# Tutorial on connecting DeepPavlov bot to Yandex.Alice
In this tutorial we show how to develop a simple bot and connect it to Yandex.Alice. Our bot is able to greet, say goodbye, and answer questions based on an FAQ list. In our case we take typical questions of MIPT entrant as an example. This list consists of 15 questions with corresponding answers with manually generated 4-6 paraphrases for each question. More details on how to build bots with DeepPavlov can be found [here](https://medium.com/deeppavlov).

## Requirements

To connect your bot to Yandex.Alice you need to have a dedicated IP address. You also need to create an ssl certificate and a key (more details in corresponding section).

First, create a virtual env with python3.6 (`source activate py36`). Then download DeepPavlov

In [ ]:
!pip install deeppavlov

## Implementation

Create skills for greeting, saying goodbye, and fallback (in case a bot doesn't have an appropriate answer).

In [1]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill

hello = PatternMatchingSkill(responses=['Привет!', 'Приветик', 'Здравствуйте', 'Привет, с наступающим Новым Годом!'],
	patterns=['Привет', 'Здравствуйте', 'Добрый день'])
bye = PatternMatchingSkill(responses=['Пока!', 'До свидания, надеюсь смог вам помочь', 'C наступающим Новым Годом!'],
	patterns=['До свидания', 'Пока', 'Спасибо за помощь'])
fallback = PatternMatchingSkill(responses=['Я не понял, но могу попробовать ответить на другой вопрос',
	'Я не понял, задайте другой вопрос'])

Download '.csv' table with questions and answers

In [2]:
import pandas as pd
faq_table = pd.read_csv('http://files.deeppavlov.ai/faq/school/faq_school.csv')
# You may also specify path to a local file instead of a link
faq_table[:10]

,Question,Answer
0,"Мы потеряли свой код регистрации, что нам делать?",Вы можете написать нам письмо на почту приемно...
1,"Утерян код регистрации. Подскажите, что с этим...",Вы можете написать нам письмо на почту приемно...
2,Где взять код регистрации?,Вы можете написать нам письмо на почту приемно...
3,Я забыл код регистрации. Как его восстановить?,Вы можете написать нам письмо на почту приемно...
4,Как восстановить код регистрации?,Вы можете написать нам письмо на почту приемно...
5,"При регистрации мы допустили ошибку, как нам п...",Некоторые данные можно поменять через личный к...
6,При регистрации была допущена ошибка. Я хотел ...,Некоторые данные можно поменять через личный к...
7,"Нужно исправить даннные, что делать?",Некоторые данные можно поменять через личный к...
8,Я ошиблась в имени ребенка. Где можно его изме...,Некоторые данные можно поменять через личный к...
9,Как изменить регистрационные данные?,Некоторые данные можно поменять через личный к...


There are several autoFAQ models in DeepPavlov. In this tutorial we use tf-idf based model.

In [3]:
from deeppavlov.core.common.file import read_json
from deeppavlov import configs
from deeppavlov import train_model

model_config = read_json(configs.faq.tfidf_autofaq)

You can specify the following fields in the config (for explanations look [here](http://docs.deeppavlov.ai/en/master/intro/config_description.html#variables)):
1. `['dataset_reader']['data_url']` - path/link to '.csv' table
2. `['dataset_reader']['x_col_name']` - name of the questions column in the table
3. `['dataset_reader']['y_col_name']` - name of the answers column in the table
4. `['metadata']['variables']['ROOT_PATH']` - path, where models will be saved

and train it.

In [4]:
model_config['dataset_reader']['data_url'] = 'http://files.deeppavlov.ai/faq/school/faq_school.csv'
model_config['dataset_reader']['x_col_name'] = 'Question'
model_config['dataset_reader']['y_col_name'] = 'Answer'
model_config['metadata']['variables']['ROOT_PATH'] = './models/'
faq = train_model(model_config)

[nltk_data] Downloading package punkt to /Users/nv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /Users/nv/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/nv/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-12-20 13:59:43.104 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /usr/local/miniconda3/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-12-20 13:59:43.193 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2018-12-20 13:59:43.206 INFO in 'deeppavlov.models.sklearn.sklearn_component'['sklearn_component'] at line 201: Loading mode

The last step is to bring all skills together using Agent.

In [5]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector

agent = DefaultAgent([hello, bye, faq, fallback], skills_selector=HighestConfidenceSelector())

Before connecting to Alice, check that everything works correctly. Agent is able to receive a list of messages.

In [6]:
print(agent(['Привет', 'У меня болит живот', 'Пока', 'бла бла бла']))

['Привет, с наступающим Новым Годом!', 'Я не понял, но могу попробовать ответить на другой вопрос', 'Пока!', 'Я не понял, но могу попробовать ответить на другой вопрос']


/usr/local/miniconda3/lib/python3.6/site-packages/deeppavlov/models/classifiers/cos_sim_classifier.py:63: RuntimeWarning: invalid value encountered in true_divide
  cos_similarities = np.array(q_vects.dot(self.x_train_features.T).todense())/norm


In order to connect to Alice we need an ssl key and a certificate. Replace `<MACHINE_IP_ADDRESS>` with actual IP address or a domain name of your server.

In [ ]:
!openssl req -new -newkey rsa:4096 -days 365 -nodes -x509 -subj "/CN=<MACHINE_IP_ADDRESS>" -keyout my.key -out my.crt

In [ ]:
from utils.alice import start_agent_server

start_agent_server(agent, host='0.0.0.0', port=5000, endpoint='/faq', ssl_key='my.key', ssl_cert='my.crt')

Now you can specify webhook (`<MACHINE_IP_ADDRESS>:5000/faq`) in [Yandex.Alice skill settings](https://dialogs.yandex.ru/developer/) and test it!


# Useful links

[DeepPavlov repository](https://github.com/deepmipt/DeepPavlov)

[DeepPavlov demo page](https://demo.ipavlov.ai)

[DeepPavlov documentation](https://docs.deeppavlov.ai)

[DeepPavlov blog](https://medium.com/deeppavlov)